In [5]:
import chess
import chess.engine
import chess.pgn
import subprocess
import os
import json
import csv
from IPython.display import display

# Paths to your engines and Maia models
stockfish_path = "/opt/homebrew/bin/stockfish"
maia_model_paths = {
    1100: "maia_weights/maia-1100.pb.gz",
    1200: "maia_weights/maia-1200.pb.gz",
    1300: "maia_weights/maia-1300.pb.gz",
    1400: "maia_weights/maia-1400.pb.gz",
    1500: "maia_weights/maia-1500.pb.gz",
    1600: "maia_weights/maia-1600.pb.gz",
    1700: "maia_weights/maia-1700.pb.gz",
    1800: "maia_weights/maia-1800.pb.gz",
    1900: "maia_weights/maia-1900.pb.gz",
}

# Initialize Stockfish engine
print("Initializing Stockfish engine...")
stockfish_engine = chess.engine.SimpleEngine.popen_uci(stockfish_path)
print("Stockfish engine initialized.")

# Initialize Maia engines
print("Initializing Maia engines...")
maia_engines = {}
for elo, path in maia_model_paths.items():
    print(f"Initializing Maia {elo} engine...")
    maia_engines[elo] = chess.engine.SimpleEngine.popen_uci(["lc0", f"--weights={path}"])
print("Maia engines initialized.")

# Function to load games from a PGN file
def load_games_from_pgn(file_path):
    games = []
    with open(file_path, 'r') as pgn_file:
        while True:
            game = chess.pgn.read_game(pgn_file)
            if game is None:
                break
            games.append(game)
    print(f"Loaded {len(games)} games from PGN file {file_path}.")
    return games

# Function to find classic puzzles in a game
def find_classic_puzzles(game, stockfish_engine, threshold=2.00):
    board = game.board()
    puzzles = []

    for move in game.mainline_moves():
        fen_before = board.fen()
        player = board.turn  # True for White, False for Black

        # Analyze the position before the move
        info = stockfish_engine.analyse(board, chess.engine.Limit(time=0.1), multipv=2)
        if len(info) < 2:
            continue

        best_move = info[0]["pv"][0]
        best_eval = info[0]["score"].relative.score(mate_score=10000)
        second_best_eval = info[1]["score"].relative.score(mate_score=10000)

        # Check if the difference is large enough
        if abs(best_eval - second_best_eval) >= threshold:
            board.push(best_move)
            if not board.is_capture(best_move):
                puzzles.append((fen_before, best_move, best_eval, second_best_eval))
                print(f"Puzzle found: {fen_before}, Best move: {best_move}, Eval difference: {abs(best_eval - second_best_eval)}")
            board.pop()

        board.push(move)

    return puzzles

# Function to evaluate Maia easiness
def evaluate_maia_easiness(fen, best_move, maia_engines):
    board = chess.Board(fen)
    easiness_scores = {}
    print(f"Evaluating Maia easiness for FEN: {fen}, Best Move: {best_move}")

    for elo, engine in maia_engines.items():
        result = engine.analyse(board, chess.engine.Limit(nodes=1))
        if "pv" in result:
            print(f"Maia {elo} PV: {result['pv']}")
            if best_move in result["pv"]:
                easiness_scores[elo] = result["pv"].index(best_move) / len(result["pv"])
            else:
                easiness_scores[elo] = None  # Assign None if the best move is not in pv
        else:
            print(f"Maia {elo} did not return PV.")
            easiness_scores[elo] = None

    return easiness_scores

# Function to sort and save results
def sort_and_save_results(puzzles, output_dir="results/classic_puzzles"):
    os.makedirs(output_dir, exist_ok=True)

    # Calculate average easiness and handle missing values
    for puzzle in puzzles:
        maia_scores = [score for score in puzzle["maia_easiness"].values() if score is not None]
        puzzle["average_easiness"] = mean(maia_scores) if maia_scores else 0.0  # Assign 0 if no scores are available

    # Sort puzzles by average easiness
    puzzles.sort(key=lambda x: x["average_easiness"], reverse=True)

    csv_file_path = os.path.join(output_dir, "classic_puzzles.csv")
    json_file_path = os.path.join(output_dir, "classic_puzzles.json")

    with open(csv_file_path, "w", newline='') as csvfile:
        csvwriter = csv.writer(csvfile)
        csvwriter.writerow(["FEN", "Best Move", "Best Eval", "Second Best Eval", "Average Easiness"])
        for puzzle in puzzles:
            csvwriter.writerow([
                puzzle["fen"],
                puzzle["best_move"].uci(),  # Convert Move object to UCI notation
                puzzle["best_eval"],
                puzzle["second_best_eval"],
                puzzle["average_easiness"]
            ])

    with open(json_file_path, "w") as jsonfile:
        json.dump(puzzles, jsonfile, indent=4)

# Load PGN files and find classic puzzles
pgn_directory = "analysis_pgns_test"
all_puzzles = []

print("Starting to find classic puzzles in PGN files...")

for pgn_file in os.listdir(pgn_directory):
    if pgn_file.endswith(".pgn"):
        pgn_file_path = os.path.join(pgn_directory, pgn_file)
        print(f"Analyzing {pgn_file_path}...")
        all_games = load_games_from_pgn(pgn_file_path)

        for game in all_games:
            print(f"Analyzing game {game.headers.get('Event', 'Unknown Event')}")
            puzzles = find_classic_puzzles(game, stockfish_engine)
            for fen, best_move, best_eval, second_best_eval in puzzles:
                maia_easiness = evaluate_maia_easiness(fen, best_move, maia_engines)
                all_puzzles.append({
                    "fen": fen,
                    "best_move": best_move,
                    "best_eval": best_eval,
                    "second_best_eval": second_best_eval,
                    "maia_easiness": maia_easiness
                })

# Sort and save results
sort_and_save_results(all_puzzles)
print("Classic puzzles analysis complete. Results saved.")


Initializing Stockfish engine...


<UciProtocol (pid=18886)>: stderr >>        _
<UciProtocol (pid=18886)>: stderr >> |   _ | |
<UciProtocol (pid=18886)>: stderr >> |_ |_ |_| v0.30.0+git.dirty built Jul 21 2023
<UciProtocol (pid=18887)>: stderr >>        _
<UciProtocol (pid=18887)>: stderr >> |   _ | |
<UciProtocol (pid=18887)>: stderr >> |_ |_ |_| v0.30.0+git.dirty built Jul 21 2023
<UciProtocol (pid=18888)>: stderr >>        _
<UciProtocol (pid=18888)>: stderr >> |   _ | |
<UciProtocol (pid=18888)>: stderr >> |_ |_ |_| v0.30.0+git.dirty built Jul 21 2023
<UciProtocol (pid=18889)>: stderr >>        _
<UciProtocol (pid=18889)>: stderr >> |   _ | |
<UciProtocol (pid=18889)>: stderr >> |_ |_ |_| v0.30.0+git.dirty built Jul 21 2023
<UciProtocol (pid=18890)>: stderr >>        _
<UciProtocol (pid=18890)>: stderr >> |   _ | |
<UciProtocol (pid=18890)>: stderr >> |_ |_ |_| v0.30.0+git.dirty built Jul 21 2023
<UciProtocol (pid=18891)>: stderr >>        _
<UciProtocol (pid=18891)>: stderr >> |   _ | |
<UciProtocol (pid=18891)>: 

Stockfish engine initialized.
Initializing Maia engines...
Initializing Maia 1100 engine...
Initializing Maia 1200 engine...
Initializing Maia 1300 engine...
Initializing Maia 1400 engine...
Initializing Maia 1500 engine...
Initializing Maia 1600 engine...
Initializing Maia 1700 engine...
Initializing Maia 1800 engine...
Initializing Maia 1900 engine...
Maia engines initialized.
Starting to find classic puzzles in PGN files...
Analyzing analysis_pgns_test/4c6eS6SH---1969 Petrosian vs. Spassky .pgn...
Loaded 4 games from PGN file analysis_pgns_test/4c6eS6SH---1969 Petrosian vs. Spassky .pgn.
Analyzing game FIDE World Championship Match 1969
Analyzing game FIDE World Championship Match 1969
Analyzing game FIDE World Championship Match 1969
Analyzing game FIDE World Championship Match 1969
Analyzing analysis_pgns_test/4S5UuAGn---1986 Karpov vs. Kasparov III.pgn...
Loaded 5 games from PGN file analysis_pgns_test/4S5UuAGn---1986 Karpov vs. Kasparov III.pgn.
Analyzing game FIDE World Champio

KeyboardInterrupt: 

In [6]:
all_puzzles

[]

In [7]:
puzzles

[]